In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['SERPAPI_KEY'] = os.getenv('SERPAPI_KEY')
serpapi_key = os.getenv("SERPAPI_KEY")

: 

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

urls = [
    "https://medium.com/@aktooall/traditional-rag-explained-from-query-to-summary-d1beef61ba8c",
    "https://medium.com/@tejpal.abhyuday/retrieval-augmented-generation-rag-from-basics-to-advanced-a2b068fd576c"
]

# Load documents from URLs
docs = []
for url in urls:
    try:
        loader = WebBaseLoader(url)
        page_docs = loader.load()
        docs.extend(page_docs)
    except Exception as e:
        print(f"❌ Failed to load {url}: {e}")


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2

)
print(llm)

## Splitting the doc into chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,
    chunk_overlap=200
)

documents = splitter.split_documents(docs)


## Creating vectorstore and storing as chunks

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
import os

# Make sure your OPENAI_API_KEY is set in environment
embedding = OpenAIEmbeddings( model="text-embedding-3-large")

# Create Chroma vector DB

vectorstore = Chroma.from_documents(documents, embedding)

# Create retriever from it
retriever = vectorstore.as_retriever()

In [ ]:
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph import add_messages

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [ ]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    name="vector_retriever",
    description="Semantic document search from internal knowledge base"
)

In [ ]:
tools = [retriever_tool]


In [ ]:
agent_with_tools = llm.bind_tools(tools)

In [ ]:
def llm_decision_maker(state: AgentState):
    message = state["messages"]
    last_message=message[-1]
    question=last_message.content
    response=agent_with_tools.invoke(question)
    return {"messages":[response]}

In [ ]:
from pydantic import BaseModel, Field

class grade(BaseModel):
    binary_score: str = Field(description="Answer 'yes' if documents are relevant to the question. Else say 'no'.")

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from typing import Literal

def grade_documents(state: AgentState) -> Literal["generator", "rewriter"]:
    print("📊 Grading retrieved docs...")
    messages = state["messages"]

    user_question = messages[0].content
    retrieved_docs = messages[-1].content

    prompt = PromptTemplate.from_template(
        "You're a helpful agent that checks if the given documents are relevant.\n"
        "Question: {question}\n"
        "Docs: {content}\n"
        "Reply only with yes or no."
    )

    llm_structured = llm.with_structured_output(grade)
    chain = prompt | llm_structured

    result = chain.invoke({"question": user_question, "content": retrieved_docs})
    score = result.binary_score.strip().lower()

    if score == "yes":
        print("✅ Docs are relevant → generator")
        return "generator"
    else:
        print("❌ Docs not relevant → rewriter")
        return "rewriter"


In [ ]:
from langchain import hub
def generate_output(state: AgentState) -> dict:
    print("📝 Generating final answer...")


    rag_prompt = hub.pull("rlm/rag-prompt")

    messages = state["messages"]
    question = messages[0].content
    docs = messages[-1].content

    chain = rag_prompt | llm
    response = chain.invoke({"question": question, "context": docs})

    return {
        "messages": [response]
    }


In [ ]:
def query_rewriter(state: AgentState) -> dict:
    print("✏️ Rewriting query...")

    from langchain_core.messages import HumanMessage
    messages = state["messages"]
    original_question = messages[0].content

    prompt = PromptTemplate.from_template(
        "Rewrite the user question for better web search results:\n\nOriginal: {question}\n\nRewritten:"
    )

    chain = prompt | llm
    rewritten = chain.invoke({"question": original_question})

    return {
        "messages": [rewritten]
    }


In [ ]:
from langchain_community.utilities.serpapi import SerpAPIWrapper

serper_search = SerpAPIWrapper(serpapi_api_key=serpapi_key)
def web_search_node(state: AgentState) -> dict:
    print("Web Search fallback via Serper")
    messages = state["messages"]
    latest_query = messages[-1].content

    try:
        search_result = serper_search.run(latest_query)
    except Exception as e:
        search_result = f"Web search failed: {e}"

    return {"messages": [HumanMessage(content=search_result)]}

In [ ]:
from langgraph.graph import StateGraph

workflow = StateGraph(AgentState)


workflow.add_node("LLM Decision Maker", llm_decision_maker)
workflow.add_node("Vector Retriever", retriever_node)
workflow.add_node("Output Generator", generate_output)
workflow.add_node("Query Rewriter", query_rewriter)
workflow.add_node("Web Search", web_search_node)

In [ ]:
from langgraph.graph import StateGraph, END,START
from langgraph.prebuilt import tools_condition


# 🔹 Entry point
workflow.add_edge(START, "LLM Decision Maker")

# 🔹 Decision Maker → Retriever OR END (based on tools_condition)
workflow.add_conditional_edges("LLM Decision Maker", tools_condition, {
    "tools": "Vector Retriever",
    END: END
})

# 🔹 Retriever → grade_documents → Generator or Rewriter
workflow.add_conditional_edges("Vector Retriever", grade_documents, {
    "generator": "Output Generator",
    "rewriter": "Query Rewriter"
})

# 🔹 Generator → END
workflow.add_edge("Output Generator", END)

# 🔹 Rewriter → Web → Decision Maker
workflow.add_edge("Query Rewriter", "Web Search")
workflow.add_edge("Web Search", "LLM Decision Maker")


In [ ]:
app=workflow.compile()

In [ ]:
app

In [ ]:
app.invoke({
    "messages": [HumanMessage(content="what is Agentic Rag")]
})
